# Deliverable 1
### Uhhh idk some stuff

In [4]:
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:
filename = "data/abp_data.csv"

df = pd.read_csv(filename)
    

In [6]:
df.head()

,permitnumber,worktype,permittypedescr,description,comments,applicant,declared_valuation,total_fees,issued_date,expiration_date,...,occupancytype,sq_feet,address,city,state,zip,property_id,parcel_id,lat,long
0,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.00,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,Comm,170.0,175 W Boundary RD,West Roxbury,MA,02132,17268,2012032000,42.260750,-71.149610
1,A1001012,OTHER,Amendment to a Long Form,Other,Amend Alt943748 to erect a roof deck as per pl...,Jusimar Oliveria,5000.00,70.0,2020-06-01 14:08:47,2020-12-01 00:00:00,...,1-3FAM,0.0,15 Prospect ST,Charlestown,MA,02129,113443,0202837000,42.375243,-71.057585
2,A1001201,INTEXT,Amendment to a Long Form,Interior/Exterior Work,Build steel balcony over garden level with sta...,Andreas Hwang,74295.75,803.0,2019-11-13 13:38:56,2020-05-13 00:00:00,...,Multi,0.0,211 W Springfield ST,Roxbury,MA,02118,129994,0402558000,42.340600,-71.080250
3,A100137,EXTREN,Amendment to a Long Form,Renovations - Exterior,Landscaping/stonework - amending permit #2801/...,NaN,15000.00,206.0,2013-01-03 14:13:09,2013-07-03 00:00:00,...,1-2FAM,0.0,14 William Jackson AVE,Brighton,MA,02135,149852,2204944000,42.344600,-71.154050
4,A1001913,INTREN,Amendment to a Long Form,Renovations - Interior NSC,Amendment to issued permit ALT888985. relocate...,Ping Mandawe,1.00,33.0,2019-10-18 09:21:00,2020-04-18 00:00:00,...,Comm,0.0,130-140 Brighton AVE,Allston,MA,02134,20073,2100846000,42.352760,-71.131850


In [7]:
applicants = df.iloc[:, [5]]
owners = df.iloc[:, [11]]

In [8]:
owners.head()

,owner
0,CITY OF BOSTON
1,RUBIO FAMILY TRUST LLC
2,LEDERMAN US REAL ESTATE CORP
3,MIARA SIMON
4,MABB LLC


In [9]:
# Since the models are based on calculations and mathematical equations, it is harder for the computers to understand texts.
# Hence, we need to encode the categorical data.
# Create an object of the LableEncoder class.
labelEncoder = LabelEncoder()

In [10]:
applicants.fillna("na")
applicants = applicants.applymap(str)
applicants

owners.fillna("na")
owners = owners.applymap(str)
owners

,owner
0,CITY OF BOSTON
1,RUBIO FAMILY TRUST LLC
2,LEDERMAN US REAL ESTATE CORP
3,MIARA SIMON
4,MABB LLC
...,...
478014,SHIGO CENTER PLAZA OWNER LLC
478015,SHIGO CENTER PLAZA OWNER LLC
478016,AVONWOOD ASSOCS LPS
478017,FORTY 6 BURROUGHS ST CONDO


In [11]:
applicants_labels = applicants.iloc[:, 0]
applicants_labels = [name.lower().replace(" ", "") for name in applicants_labels]
# applicants_map = dict()
# for i in range(len(owners)):
#     applicants_map[applicants_labels[i]] = applicants.iloc[:, 0][i]
applicants_labels = labelEncoder.fit_transform(applicants_labels)
# applicants_labels.head()
# applicants.dtypes
applicants_labels = pd.DataFrame(applicants_labels)
applicants_labels

applicants_labels = pd.DataFrame(applicants_labels)
applicants_labels_with_names = applicants_labels.copy()
applicants_labels_with_names['names'] = applicants

applicants_labels_with_names

,0,names
0,30653,Renee Santeusanio
1,20069,Jusimar Oliveria
2,1256,Andreas Hwang
3,27060,nan
4,30052,Ping Mandawe
...,...,...
478014,5505,Christine McMahon
478015,5505,Christine McMahon
478016,30604,Regina Olivieri
478017,23488,Maria Dubrowski


In [12]:
applicants_labels_vals = applicants_labels.value_counts()
# applicants_labels_vals
index = [tup[0] for tup in applicants_labels_vals.index]
# index

In [13]:
# applicants_labels.value_counts().index
applicants_labels_vals.index = labelEncoder.inverse_transform(index)
pd.DataFrame(applicants_labels_vals)

,0
nan,14215
roberttrethewey,5892
johnmclaughlin,3603
paulguarracino,3253
richardfallone,3059
...,...
kevinlaik,1
kevinlang,1
kevinlarnach,1
kevinlau,1


In [14]:
owners_labels = owners.iloc[:, 0]
owners_labels = [name.lower().replace(" ", "") for name in owners_labels]
# owners_map = dict()
# for i in range(len(owners)):
#     owners_map[owners_labels[i]] = owners.iloc[:, 0][i]
owners_labels = labelEncoder.fit_transform(owners_labels)

owners_labels = pd.DataFrame(owners_labels)
owners_labels_with_names = owners_labels.copy()
owners_labels_with_names['names'] = owners

owners_labels_with_names

,0,names
0,12221,CITY OF BOSTON
1,48278,RUBIO FAMILY TRUST LLC
2,32287,LEDERMAN US REAL ESTATE CORP
3,37460,MIARA SIMON
4,34217,MABB LLC
...,...,...
478014,50819,SHIGO CENTER PLAZA OWNER LLC
478015,50819,SHIGO CENTER PLAZA OWNER LLC
478016,5524,AVONWOOD ASSOCS LPS
478017,21276,FORTY 6 BURROUGHS ST CONDO


In [15]:
owners_labels_vals = owners_labels.value_counts()
owners_index = [tup[0] for tup in owners_labels_vals.index]

In [16]:
owners_labels_vals.index = labelEncoder.inverse_transform(owners_index)
owners_labels_vals

nan                         36903
cityofboston                 6003
bostonhousingauthority       2633
marriottownershipresorts     2546
northeasternuniversity       2523
                            ...  
leachdaphney                    1
twenty1marylandst               1
leabokarld                      1
le-royangusjetal                1
celestineanthony                1
Length: 62285, dtype: int64

In [17]:
df1 = df[applicants['applicant'].str.contains("Robert Trethewey")]
df1.drop_duplicates("owner").iloc[:, [5, 11]]
# df1

,applicant,owner
67609,Robert Trethewey,OCONNOR CAROL A
188501,Robert Trethewey,EGAN PATRICK ROBERT
189588,Robert Trethewey,ALJOE NICOLE N
204901,Robert Trethewey,SEVENTY 2-74 EAST DEDHAM
204954,Robert Trethewey,WEDIKO CHILDRENS SERVICES IN
...,...,...
340714,Robert Trethewey,HORENSTEIN MARK
344617,Robert Trethewey,BELGRADE AND BIRCH LLC
345559,Robert Trethewey,ENGLAND ALEXANDRA
345754,Robert Trethewey,SIXTEEN GLADE AVENUE
